<a href="https://colab.research.google.com/github/jsansao/transformers_pt/blob/main/BR_BERTo_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferência usando transformers pré-treinados do HuggingFace 

In [1]:
#@title Passo 1:Instalando Hugging Face Transformers
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.9.1
# tokenizers version at notebook update --- 0.7.0

Found existing installation: tensorflow 2.8.2+zzzcolab20220719082949
Uninstalling tensorflow-2.8.2+zzzcolab20220719082949:
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220719082949
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-28_9xtcn
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-28_9xtcn
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.0 MB 27.9 MB/s 
     |████████████████████████████████| 120 kB 60.9 MB/s 
  Created wheel for transformers: filename=transformers-4.23.0.dev0-py3-none-any.whl size=4993773 sha256=f12a6f5a5e65fbfb475dc9e0d2a772077d893c3e2dff365c19b07dbe6410f043
  Stored in directory: /tmp/pip-ephem-wheel-cache-4iztj9nr/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa

In [2]:
#@title Step 1: Loading the Dataset
#1.Load kant.txt using the Colab file manager
#2.Downloading the file from GitHub
#!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/dump_Machado_Nobreak.txt --output "kant.txt"
!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/Lyrics_ChicoBuarque.txt --output "kant.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  251k  100  251k    0     0  1613k      0 --:--:-- --:--:-- --:--:-- 1613k


In [3]:
#@title Passo 2:Baixando e salvando BR_BERTo
#https://huggingface.co/rdenadai/BR_BERTo

from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("rdenadai/BR_BERTo")

model = AutoModelForMaskedLM.from_pretrained("rdenadai/BR_BERTo")

Downloading:   0%|          | 0.00/516 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/695M [00:00<?, ?B/s]

In [4]:
#@title Step 10: Building the Dataset
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./kant.txt",
    block_size=32,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 969 ms, sys: 491 ms, total: 1.46 s
Wall time: 1.64 s


In [5]:
#@title Step 11: Defining a Data Collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [6]:
#@title Step 12: Initializing the Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./KantaiBERT",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [7]:
#@title Step 13: Pre-training the Model
%%time
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9957
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3120
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,5.442600
1000,4.542500
1500,4.184400
2000,3.908200
2500,3.726600
3000,3.565000




Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 11min 39s, sys: 3min 1s, total: 14min 41s
Wall time: 14min 42s


TrainOutput(global_step=3120, training_loss=4.19829963292831, metrics={'train_runtime': 882.8858, 'train_samples_per_second': 112.778, 'train_steps_per_second': 3.534, 'total_flos': 645803867338560.0, 'train_loss': 4.19829963292831, 'epoch': 10.0})

In [8]:
#@title Step 14: Saving the Final Model(+tokenizer + config) to disk
trainer.save_model("./KantaiBERT")

Saving model checkpoint to ./KantaiBERT
Configuration saved in ./KantaiBERT/config.json
Model weights saved in ./KantaiBERT/pytorch_model.bin


In [10]:
#@title Step 3: Configurando o pipeline fill-mask
#@title Step 15: Language Modeling with the FillMaskPipeline
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./KantaiBERT",
    tokenizer=tokenizer
)

loading configuration file ./KantaiBERT/config.json
Model config RobertaConfig {
  "_name_or_path": "./KantaiBERT",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 8,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.23.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 150000
}

loading configuration file ./KantaiBERT/config.json
Model config RobertaConfig {
  "_name_or_path": "./KantaiBERT",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  

In [16]:
fill_mask("Apesar de <mask> ")

[{'score': 0.38974058628082275,
  'token': 2139,
  'token_str': ' mim',
  'sequence': 'Apesar de mim '},
 {'score': 0.12209956347942352,
  'token': 582,
  'token_str': ' você',
  'sequence': 'Apesar de você '},
 {'score': 0.06614171713590622,
  'token': 1612,
  'token_str': ' amor',
  'sequence': 'Apesar de amor '},
 {'score': 0.018230635672807693,
  'token': 942,
  'token_str': ' tudo',
  'sequence': 'Apesar de tudo '},
 {'score': 0.015597295947372913,
  'token': 884,
  'token_str': ' ti',
  'sequence': 'Apesar de ti '}]